In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.cluster_analysis import prep_cluster_analysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.svm import LinearSVC


In [2]:
length_per_window = 10
num_sliding_windows = int(1000/ length_per_window)

In [ ]:
participants = ["904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]

In [3]:
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]


In [ ]:
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124",\
               "904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]


In [4]:
#Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)
Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials", length_per_window=length_per_window)
#Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants", length_per_window=length_per_window)


loaded


In [ ]:
Xwin[0].shape

In [ ]:
ywin[0].shape

In [ ]:
num_sliding_windows

In [ ]:
# # Cross validation

#X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)

# for j in range(num_sliding_windows):
#     #model = SVC(gamma=.001, kernel = 'rbf', C=1000)
#     model = LinearSVC(C=1e-9, max_iter=5000)
#     cv_results = cross_validate(model, X[j], y[j], cv=num_folds)
#     if j in results.keys(): 
#         results[j] += cv_results['test_score'].tolist()
#     else:
#         results[j] = cv_results['test_score'].tolist()
    

# for i in range(num_sliding_windows):
#     assert len(results[i]) == num_iterations * num_folds

In [5]:
# Cross validation with RepeatedKFold

num_iterations = 3
num_folds = 5


import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score



results = {}
for j in range(num_sliding_windows):
    X = Xwin[j]
    y = ywin[j]

    #model = SVC(gamma=.001, kernel = 'rbf', C=1e-4)
    model = LinearSVC(C=1e-5, max_iter=5000)
    rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_iterations, random_state=2652124)
    for train_index, test_index in rkf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        testScore = accuracy_score(y_test,y_pred)

        if j in results.keys(): 
            results[j].append(testScore)
        else:
            results[j]=[]
            results[j].append(testScore)
    

for i in range(num_sliding_windows):
    assert len(results[i]) == num_iterations * num_folds

In [6]:
X.shape

(208, 600)

In [7]:
X_train.shape

(167, 600)

In [8]:
scoreMean = []
stdev = []

for i in range(num_sliding_windows):
    scoreMean.append(np.mean(results[i]))
    stdev.append(np.std(results[i]))



In [9]:
np.mean(scoreMean)

0.47284204413472714

In [10]:
scoreMean

[0.4771970576848626,
 0.46918312040263266,
 0.5046457607433216,
 0.46136275648470776,
 0.47921022067363533,
 0.5481997677119629,
 0.5274874177313201,
 0.5067363530778165,
 0.5197831978319783,
 0.5225706542779712,
 0.4887340301974449,
 0.41982191250483947,
 0.42613240418118475,
 0.47402245451025943,
 0.4694154084397988,
 0.4567557104142471,
 0.49194734804490914,
 0.5110336817653891,
 0.4788617886178862,
 0.47413859852884255,
 0.4696476964769648,
 0.48393341076267915,
 0.48552071234998073,
 0.5017034456058848,
 0.5127758420441348,
 0.4854045683313977,
 0.4789779326364693,
 0.4597754548974062,
 0.46647309330236164,
 0.4535036778939218,
 0.4548974061169184,
 0.46438250096786693,
 0.47746806039488976,
 0.4902439024390245,
 0.4694928377855208,
 0.4597367402245452,
 0.4757646147890051,
 0.4903213317847464,
 0.5015098722415796,
 0.4903600464576075,
 0.5128532713898567,
 0.48710801393728237,
 0.46461478900503306,
 0.4775454897406118,
 0.4532713898567558,
 0.45199380565234226,
 0.471118854045683

In [11]:
#stdev

In [12]:
# T-test
accuracy_by_guessing = [0.5] * (num_iterations * num_folds)
pvalues = []
for i in range(num_sliding_windows):
    istat = stats.ttest_1samp(results[i], .5)
    pvalues += [istat.pvalue] if istat.statistic > 0 else [1]

In [13]:
# Finding contiguous time cluster
valid_window = [i for i,v in enumerate(pvalues) if v <= 0.025]
print(valid_window)

[5]
